<a href="https://colab.research.google.com/github/silasp/AdaptiveResize/blob/master/Silas_Video_Scale_and_Frame_Interpolate_Real_ESRGAN_(anime)_and_RIFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video upscaling and frame interpolation tool

There was nothing super reliable available, so I cobbled this together.

Tested on colab+ / google drive. You'll need both of these for it to work, probably.

From:
- Real-ESRGAN [GitHub repo](https://github.com/xinntao/Real-ESRGAN). 
- RIFE [GitHub repo](https://github.com/megvii-research/ECCV2022-RIFE).


# ESRGAN prep
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment.

In [1]:
## See what GPU you got
!nvidia-smi

Mon Oct  3 06:36:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Params
working_dir = '/content/gdrive/MyDrive/' #@param {type:"string"}
original_video = 'Gs6.mp4' #@param {type:"string"}
x2_video = 'Gs6_x2.mp4' #@param {type:"string"}
hifps_x2_video = 'Gs6_x2_hifps.mp4' #@param {type:"string"}

#@markdown Interpolate steps: 1=2x, 2=4x, 3=8x, 4=16x (approx)
#@markdown <br> For example, 3 will convert a 5fps video to 40fps.
interpolate_steps=3#@param {type:"slider", min:1, max:5, step:1}


## video_path = 'upload/Gs6.mp4' #@param {type:"string"}

video_path = working_dir + original_video
x2_video_path = working_dir + x2_video
hifps_x2_video_path = working_dir + hifps_x2_video


#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
# Clone Real-ESRGAN, install dependencies

# Clone to gdrive if not there already
import os
if os.path.isdir('/content/gdrive/MyDrive/Real-ESRGAN'):
  %cd /content/gdrive/MyDrive/Real-ESRGAN
elif os.path.isdir('/content/gdrive/MyDrive/'):
  %cd /content/gdrive/MyDrive
  !git clone https://github.com/xinntao/Real-ESRGAN.git
  %cd Real-ESRGAN

# Set up the environment
print("Started Installing libraries for Real-ESRGAN upscaling.")
!pip -q install basicsr
!pip -q install facexlib
!pip -q install gfpgan
!pip -q install ffmpeg-python
!pip -q install -r requirements.txt
print("Running setup.py - this will take a while...")
!python setup.py develop &> /dev/null
print("Finished Installing libraries for Real-ESRGAN upscaling.")

/content/gdrive/MyDrive/Real-ESRGAN
Started Installing libraries for Real-ESRGAN upscaling.
     |████████████████████████████████| 776.3 MB 11 kB/s 
     |████████████████████████████████| 19.1 MB 22.8 MB/s 
running develop
running egg_info
writing realesrgan.egg-info/PKG-INFO
writing dependency_links to realesrgan.egg-info/dependency_links.txt
writing requirements to realesrgan.egg-info/requires.txt
writing top-level names to realesrgan.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'realesrgan.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/realesrgan.egg-link (link to .)
realesrgan 0.3.0 is already the active version in easy-install.pth

Installed /content/gdrive/MyDrive/Real-ESRGAN
Processing dependencies for realesrgan==0.3.0
Searching for tqdm==4.64.1
Best match: tqdm 4.64.1
Adding tqdm 4.64.1 to easy-install.pth file
Installing tqdm script to /usr/local/bin

Using /usr/lo

# 5. Upscale and save upscaled video to x2_video_path


In [20]:
print("Starting video upscale...")
# ! python inference_realesrgan_video.py -i upload/onepiece_demo.mp4 -n realesr-animevideov3 -s 2 --suffix outx2
! python inference_realesrgan_video.py -i $video_path -n realesr-animevideov3 -s 2 --suffix outx2
 # Argements
 # -i, --input: input video
 # -n, --model_name: Used model name
 # -s, --outscale: Scale
 # -suffix: Suffix of the output video



inference: 100% 909/909 [04:40<00:00,  3.24frame/s]


In [29]:
 # Move to specified path

x2_workingfile_path = '/content/gdrive/MyDrive/Real-ESRGAN/results/*_outx2.mp4'

!ls $x2_workingfile_path
!mv $x2_workingfile_path $x2_video_path 

/content/gdrive/MyDrive/Real-ESRGAN/results/Gs6_outx2.mp4


# Interpolate Frames with RIFE

In [30]:
## Install RIFE

%cd /content
!git clone https://github.com/hzwer/arXiv2020-RIFE

/content
fatal: destination path 'arXiv2020-RIFE' already exists and is not an empty directory.


In [31]:
## Download the interpolation AI model 

!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e -aoa RIFE_trained_model_v3.6.zip

mkdir: cannot create directory ‘/content/arXiv2020-RIFE/train_log’: File exists
/content/arXiv2020-RIFE/train_log
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
To: /content/arXiv2020-RIFE/train_log/RIFE_trained_model_v3.6.zip
100% 11.3M/11.3M [00:00<00:00, 291MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 11332064 bytes (11 MiB)

Extracting archive: RIFE_trained_model_v3.6.zip
--
Path = RIFE_trained_model_v3.6.zip
Type = zip
Physical Size = 11332064

  0%    
Would you like to replace the 

In [32]:
%cd /content/arXiv2020-RIFE/
## Demo video not needed
## !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 -q install -r requirements.txt
print("You can safely ignore dependency errors for torchtext, torchaudio, gfpgan, fastai, basicsr and realesrgan - it works just fine without these...")

/content/arXiv2020-RIFE
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.6.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.6.0 which is incompatible.
gfpgan 1.3.8 requires torch>=1.7, but you have torch 1.6.0 which is incompatible.
fastai 2.7.9 requires torch<1.14,>=1.7, but you have torch 1.6.0 which is incompatible.
fastai 2.7.9 requires torchvision>=0.8.2, but you have torchvision 0.7.0 which is incompatible.
basicsr 1.4.2 requires torch>=1.7, but you have torch 1.6.0 which is incompatible.
realesrgan 0.3.0 requires torch>=1.7, but you have torch 1.6.0 which is incompatible.


In [35]:
!ls '/content/arXiv2020-RIFE/'
!cp $x2_video_path '/content/arXiv2020-RIFE/video.mp4'
!ls '/content/arXiv2020-RIFE/'


benchmark	  docker	      model		train.py
Colab_demo.ipynb  inference_img.py    README.md
dataset.py	  inference_video.py  requirements.txt
demo		  LICENSE	      train_log
benchmark	  docker	      model		train.py
Colab_demo.ipynb  inference_img.py    README.md		video.mp4
dataset.py	  inference_video.py  requirements.txt
demo		  LICENSE	      train_log


In [36]:
!python3 inference_video.py --help

print("Starting video interpolation...")
!python3 inference_video.py --exp=$interpolate_steps --video=video.mp4

Loaded v3.x HD model.
video.mp4, 909.0 frames in total, 5.0FPS to 40.0FPS
The audio will be merged after interpolation process
100% 908/909.0 [13:15<00:00,  1.14it/s]
[mpeg4 @ 0x55b1600] Invalid pts (7264) <= last (7264)


In [41]:
!ls '/content/arXiv2020-RIFE/'

## Copy back to gdrive
!cp /content/arXiv2020-RIFE/video*fps.mp4 $hifps_x2_video_path

print("All done - if it worked, processed video should be available at: " + hifps_x2_video_path)

benchmark	  docker	      model		train.py
Colab_demo.ipynb  inference_img.py    README.md		video_8X_40fps.mp4
dataset.py	  inference_video.py  requirements.txt	video.mp4
demo		  LICENSE	      train_log
All done - if it worked, processed video should be available at: /content/gdrive/MyDrive/Gs6_x2_hifps.mp4
